In [1]:
pip install openai requests


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openai
import re
import json
import requests
import os

openai.api_base = "https://oai.hconeai.com/v1"
# uncomment the next line with your own openai api key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
def get_completion(prompt, model="gpt-4", stop=None):
    messages = [{"role":"user","content":prompt}]
    response = openai.ChatCompletion.create(
        model = model,
        messages = messages,
        temperature = 0,
        stop = stop,
    )
    return response.choices[0].message.content

In [4]:
get_completion("tell me a funny joke!")

"Why don't scientists trust atoms? Because they make up everything!"

In [5]:
class Tool():
    def __init__(self, name, description, prompt):
        self.name = name
        self.description = description

    def run(self, tool_input):
        pass

In [6]:
import sys
from io import StringIO
class PythonREPLTool(Tool):
    def __init__(self):
        self.name = "Python REPL"
        self.description = """
        A Python shell. Use this to execute python commands. 
        Input should be a valid python program.
        If you want to see the output of a value, you should print it out with `print(...).`
        """
    
    def run(self, tool_input):
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            exec(tool_input.strip().strip("```python").strip("```"), globals(), None)
            sys.stdout = old_stdout
            output = mystdout.getvalue()
        except Exception as e:
            sys.stdout = old_stdout
            output = str(e)
        return output
        


In [7]:
class Agent():
    def __init__(self, tools):
        self.prompt_template = """Answer the question as best as you can using the following tools: 

{tool_description}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, exactly one element of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation repeats N times, use it until you are sure of the answer)
Thought: I now know the final answer
Final Answer: your final answer to the original input question

Begin!

Question: {question}
"""
        self.tools = tools

    def run(self, question):
        prompt = self.prompt_template.format(
            tool_description= "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools]),
            tool_names = ",".join([tool.name for tool in self.tools]),
            question = question,
        )
        final_answer = None
        while not final_answer:
            response = get_completion(prompt, stop=['Observation','\nObservation'])
            tool, tool_input, final_answer = self.__parse(response)
            if final_answer:
                break
            else:
                tool_by_names = {tool.name: tool for tool in self.tools}
                tool_result = tool_by_names[tool].run(tool_input)
                prompt += response
                prompt += f"\nObservation: {tool_result}\n"
                print(prompt)
                
        return final_answer
        
        
    def __parse(self, response):
        # return Action, Action Input, Final Answer
        if "Final Answer" in response:
            return None, None, response.split("Final Answer:")[-1].strip()
        else:
            regex = r"Action: [\[]?(.*?)[\]]?[\n]*Action Input:[\s]*(.*)"
            match = re.search(regex, response, re.DOTALL)
            if not match:
                raise ValueError(f"Output of LLM is not parsable for next tool use: `{response}`")
            tool = match.group(1).strip()
            tool_input = match.group(2)
            return tool, tool_input, None
        
        


In [8]:
codeAgent = Agent(tools=[PythonREPLTool()])
codeAgent.run("Show me all the prime numbers < 100")

Answer the question as best as you can using the following tools: 

Python REPL: 
        A Python shell. Use this to execute python commands. 
        Input should be a valid python program.
        If you want to see the output of a value, you should print it out with `print(...).`
        

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, exactly one element of [Python REPL]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation repeats N times, use it until you are sure of the answer)
Thought: I now know the final answer
Final Answer: your final answer to the original input question

Begin!

Question: Show me all the prime numbers < 100
Thought: I need to write a function to check if a number is prime and then use it to find all prime numbers less than 100.
Action: Python REPL
Action Input: 
```python
def is_prim

'The prime numbers less than 100 are [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97].'

In [9]:
class ClickUpTool(Tool):
    def __init__(self):
        self.name = "Click up"
        self.description = """
        A tool to create new tasks in the click up app. 
        Input must be a list of valid json objects in this format {name: the name of the new task, description: the description of the task}.
        Here's an example [{"name": "cool task", "description": "cool description"}]
        """
        self.api_key = os.getenv('CLICKUP_API_KEY')
        self.list_id = os.getenv('CLICKUP_LIST_ID')

        
    def run(self, tool_input):
        input_array = json.loads(tool_input.strip().strip("```"))
        result = []
        for task in input_array:
            r = requests.post(f"https://api.clickup.com/api/v2/list/{self.list_id }/task" ,data=task, headers={"Authorization":self.api_key})
            result.append((r.status_code, r.reason))
        return result

In [10]:
clickupAgent = Agent(tools=[PythonREPLTool(), ClickUpTool()])
clickupAgent.run("generate 5 ideas for Podium's AI hackathon, then add them as new task to click up")

Answer the question as best as you can using the following tools: 

Python REPL: 
        A Python shell. Use this to execute python commands. 
        Input should be a valid python program.
        If you want to see the output of a value, you should print it out with `print(...).`
        
Click up: 
        A tool to create new tasks in the click up app. 
        Input must be a list of valid json objects in this format {name: the name of the new task, description: the description of the task}.
        Here's an example [{"name": "cool task", "description": "cool description"}]
        

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, exactly one element of [Python REPL,Click up]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation repeats N times, use it until you are sure of the answer)
Thought: I now know t

"I have generated 5 ideas for Podium's AI hackathon and added them as new tasks to Click up."